# Playground

This is a testing notebook for thinking about HTCondor job submission from Python.

---

This version moves away from the "job" word. You can define a "task" which represents the execution of a user-provided function. The function must take exactly two arguments: an input file path, and an output file path. The user should read input data from the input file and write output data to the output file.

The task can the be "submitted". Behind the scenes, this makes an HTCondor job. The task tracks its own state, and provides the output file produced by the user's function as a member variable when it is done executing.

---

Start by importing various Python standard library packages that will be used later, as well as the `Task` class and `TaskState` enum.

In [1]:
from pathlib import Path
import time

from htcondor_job import Task, TaskState

Our goal is to square a number using this function:

In [2]:
def square(x):
    return x ** 2

In [3]:
square(2)

4

`Task` will eventually need to be handed a function that takes two arguments: the path to the input file, and the path to the output file. You there need to write a "wrapper" function which read your inputs from the input file, and writes your output to the output file.

In this case, we will simply read and write the numbers to plain-text files. The input and output paths will be given as `pathlib.Path` objects, so we can use their `write_text` and `read_text` methods. (`pathlib` docs: https://docs.python.org/3/library/pathlib.html#module-pathlib)

In [4]:
def square_wrapper(input_file, output_file):
    print(f'input  file is {input_file}')
    print(f'output file is {output_file}')
    
    input_number = int(input_file.read_text())  # read the input number from the file, convert string to an actual integer
    
    output_number = square(input_number)        # run our function
    
    output_file.write_text(str(output_number))  # must convert output number to a string for writing plain text

Make an input file containing the number 2:

In [5]:
input_file = Path.cwd() / 'input_file'
input_file.write_text(str(2));

We can test our wrapper locally:

In [6]:
local_output = Path.cwd() / 'local_output'

square_wrapper(input_file, local_output)
output_number = int(local_output.read_text())
print(f'output number is {output_number}')

input  file is /home/jovyan/htcondor-job/input_file
output file is /home/jovyan/htcondor-job/local_output
output number is 4


Make a "task", which needs to know what function to run and where the input file is. The function to run is `square_wrapper`, and the input file is the `input_file` we made above.

In [7]:
task = Task(
    function = square_wrapper,
    input_file = input_file,
)
task

Task [TaskState.Unsubmitted] square_wrapper(/home/jovyan/htcondor-job/input_file)

Submit the task. HTCondor will schedule it for execution.

In [8]:
task.submit()

Task [TaskState.Idle] square_wrapper(/home/jovyan/htcondor-job/input_file)

The state of a `Task` is available through the attribute `Task.state`. This attribute will be updated in the background for you.

In [9]:
possible_states = "\n  ".join(str(t) for t in TaskState)
print(f'The possible task states are:\n  {possible_states}\n')
print(f'The current state of task is {task.state}')

The possible task states are:
  TaskState.Unsubmitted
  TaskState.Idle
  TaskState.Running
  TaskState.Submitted
  TaskState.Held
  TaskState.Completed
  TaskState.Removed

The current state of task is TaskState.Idle


Wait for completion:

In [10]:
while task.state is not TaskState.Completed:
    print(task.state)
    time.sleep(1)
print(task.state)   # print out the final state

TaskState.Idle
TaskState.Idle
TaskState.Idle
TaskState.Idle
TaskState.Idle
TaskState.Completed


Read the task's output file:

In [11]:
int(task.output_file.read_text())

4